# Stamen Toner style

In [42]:
import os
import json
import folium
import pandas as pd
from geopy.geocoders import Nominatim

stamen_map = folium.Map(location=[22.247500, 114.175724], tiles='Stamen Toner',
                   zoom_start=17)
folium.Marker(location=[22.248626, 114.174342], popup='The MTR station').add_to(stamen_map)
folium.CircleMarker(location=[22.246660, 114.175785], radius=100,
                    popup='Ocean Park', color='#3186cc',
                    fill_color='#3186cc').add_to(stamen_map)

stamen_map.save(os.path.join('results', 'portland.html'))
stamen_map

In [2]:
m = folium.Map(location=[22.247500, 114.175724], zoom_start=17)

marker_cluster = folium.MarkerCluster().add_to(m)


folium.Marker(
    location=[22.248626, 114.174342],
    popup='Ocean Park Station',
    icon=folium.Icon(color='green', icon='ok-sign'),
).add_to(marker_cluster)

folium.Marker(
    location=[22.246660, 114.175785],
    popup='Ocean Park',
    icon=folium.Icon(color='red', icon='remove-sign'),
).add_to(marker_cluster)


m.save(os.path.join('results', 'markerCluster.html'))
m

In [4]:
df = pd.read_csv("./data/CCI Estate for Opendata.csv")
nom = Nominatim()

df["Address"] = df["e_estate"] 
df["Coordinates"] = df["Address"].apply(nom.geocode)
df["latitude"] = df["Coordinates"].apply(lambda x: x.latitude if x != None else None)
df["longitude"] = df["Coordinates"].apply(lambda x: x.longitude if x != None else None)
df.to_csv("./data/CCI_Estate_Modified.csv", encoding="utf-8", index=False)
df[:3]

,c_estate,e_estate,pc_addr,pe_addr,scp_mktc,scp_mkte,min_opdate,max_opdate,blgcount,pc_dev,pe_dev,facilities_c,facilities_e,url,Address,Coordinates,latitude,longitude
0,碧堤半島,BELLAGIO,青山公路33號,33 CASTLE PEAK ROAD,新界西,NW,2003,2006,8,會德豐/九龍倉,WHEELOCK/THE WHARF,室外泳池、兒童遊樂場 / 兒童遊戲室、健身室、按摩池、桑拿浴室、蒸氣浴室、乒乓球室、壁球場、...,Swimming Pool、Children Playground/Play Room、Gy...,http://hk.centadata.com/ptest.aspx?type=3&code...,BELLAGIO,"(Bellagio, CO, LOM, Italia, (45.9872549, 9.261...",45.987255,9.26130
1,青怡花園,TSING YI GARDEN,青綠街7-19號,7-19 TSING LUK STREET,新界西,NW,1986,1986,7,長江,CK HUTCHISON,室外泳池、遊樂場 / 遊戲室、網球場、羽毛球場、緩跑徑,Swimming Pool、Playground/Playroom、Tennis Court...,http://hk.centadata.com/ptest.aspx?type=2&code...,TSING YI GARDEN,"(青怡花園 Tsing Yi Garden, 青衣 Tsing Yi, 青裕新村 Tsing...",22.353739,114.10509
2,翠怡花園,GREENFIELD GARDEN,楓樹窩路1號,1 FUNG SHUE WO ROAD,新界西,NW,1989,1990,11,新鴻基/捷和,SUN HUNG KAI/CHIAP HUA,室外泳池、兒童嬉水池、兒童遊樂場 / 兒童遊戲室、健身室、桑拿浴室、乒乓球室、壁球場、網球場...,Swimming Pool、Children's Pool、Children Playgro...,http://hk.centadata.com/ptest.aspx?type=3&code...,GREENFIELD GARDEN,"(2-20, 榮富苑 Greenfield Garden, 旺角 Mong Kok, 油尖旺...",22.319466,114.16552


In [5]:
e_df = pd.read_csv("./data/CCI_Estate_Modified.csv", encoding="utf-8")

In [7]:
estat_map = folium.Map(location=[22.396428, 114.109497], zoom_start=10)
estat_marker_cluster = folium.MarkerCluster().add_to(estat_map)

# There is a inline if just like the other language of like true?false: true-statment
# When we loop more than one iterator, Please use zip()
for lat, lon, count, e_estate in zip(e_df["latitude"], e_df["longitude"], e_df["blgcount"],e_df["e_estate"]):

    folium.Marker(
        location=[lat, lon],
        popup= e_estate,
        icon=folium.Icon(
            color='green' if count in range(0, 15) else "red" ,
            icon='ok-sign'),
    ).add_to(estat_marker_cluster)
    
estat_map

In [8]:
zoom_map = estat_map;
zoom_map.zoom_start = 16
zoom_map.location = [22.306114, 114.187945976585]
zoom_map

## Choropleth map

By using [adc4gis](https://ogre.adc4gis.com/) you can convert ogr file(Shape files) to GeoJSON file which is he format that folium can read. <br/>
Then set soruce SRS and target SRS to "EPSG:4326".
> The shapefile format is a popular geospatial vector data format for geographic information system (GIS) software. For more details you may check [Shapefile](https://en.wikipedia.org/wiki/Shapefile).

GeoJSON is a standard JSON strusture that especially handling Geographic Information System(GIS) information. One GeoJSON object can be represented by geometry structure such as Point, LineString and Polygon. Also, it is a set of Feature. or a series of features (FeatureCollection).

> Feature classes are homogeneous collections of common features, each having the same spatial representation, such as points, lines, or polygons, and a common set of attribute columns, for example, a line feature class for representing road centerlines. 

## School Attendance Rates by District Council District

GeoJSON by [HK-geojson](https://github.com/2blam/HK-geojson)<br/><br/>
School Attendance Rates by District Council District, Sex and Age Group, 2011 (B207)by [Data.gov.hk](https://data.gov.hk/tc-data/dataset/hk-censtatd-census-2011-population-census-main-tables)

In [122]:
choropleth_map = folium.Map(location=[22.306428, 114.180497], zoom_start=12)
hk_geoJSON = os.path.join('data', 'polygon.json')
geo_json_data = json.load(open(hk_geoJSON,encoding = 'utf8') , encoding="utf-8")

# folium.GeoJson(geo_json_data).add_to(tmp_estat_map)

df_att_rate = pd.read_csv( os.path.join('data','SchoolAttendanceRates.csv'))

choropleth_map.choropleth(
    geo_str=geo_json_data,
    data = df_att_rate,
    columns=['CACODE', 'Rate'],
    threshold_scale=[ 18,20, 21, 22, 23,25],
    key_on='feature.properties.CACODE',
    fill_color='BuPu',
    fill_opacity=0.7, 
    line_opacity=0.8,
    legend_name='School Attendance Rates by District Council District (%)',
    reset = True
)
choropleth_map